<a href="https://colab.research.google.com/github/meetarora10/veersa-hack/blob/main/model_assigning_doctor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [79]:
!pip install gtts
!pip install openai-whisper googletrans

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 25.8 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803404 sha256=0751365e8701f776d93eadbfb4513f1daef5cdc56ce953146b162748350aa472
  Stored in directory: /root/.cache/pip/wheels/2f/f2/ce/6eb23db4091d026238ce76703bd66da60b969d70bcc81d5d3a
Successfully built openai-whisper
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency co

In [1]:
from gtts import gTTS

# Hindi text
text = "मुझे बुखार है कल से"

# Convert to speech
tts = gTTS(text=text, lang='hi')
tts.save("mujhe_bukhar_hai_kal_se.mp3")
print("Audio saved as mujhe_bukhar_hai_kal_se.mp3")


Audio saved as mujhe_bukhar_hai_kal_se.mp3


In [10]:
import whisper
#import spacy
import asyncio
# 1. Load Whisper model
model = whisper.load_model('medium') # or "large" if you want
 # or "large" if you want

# 2. Transcribe audio file
result = model.transcribe('/content/mujhe_bukhar_hai_kal_se.mp3')
transcribed_text = result["text"]
print("Transcribed text:", transcribed_text)
from googletrans import Translator

# translator = Translator()
# translated_text = translator.translate(transcribed_text, src='hi', dest='en')
# print("Translated text:", translated_text)

async def translate_and_continue(text):
    translator = Translator()
    translation = await translator.translate(text, src='hi', dest='en')
    print("Translated text:", translation.text)

    # You can pass translation.text to your NER model here
    # entities = nlp_pipeline(translation.text)
    # print(entities)

# This works in Colab or Jupyter
await translate_and_continue(transcribed_text)

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcribed text:  मुझे बुखार है कल से
Translated text: I have fever from tomorrow


In [13]:
!pip install deep_translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.0 MB/s eta 0:00:00


In [20]:
from deep_translator import GoogleTranslator

translated_text = GoogleTranslator(source='auto', target='en').translate(transcribed_text)
print("Translated text:", translated_text)

# Now use this in your NER pipeline
entities = nlp(translated_text)
print(entities)


Translated text: I have fever from tomorrow
[{'entity_group': 'Sign_symptom', 'score': np.float32(0.9999182), 'word': 'fever', 'start': 7, 'end': 12}, {'entity_group': 'Time', 'score': np.float32(0.703359), 'word': 'tomorrow', 'start': 18, 'end': 26}]


In [19]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# Replace with your chosen model
model_name = "d4data/biomedical-ner-all"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

text = "The patient is having stress and is depressed."

entities = nlp(text)
for ent in entities:
    print(f"{ent['word']} ({ent['entity_group']}) - Score: {ent['score']:.2f}")


Device set to use cpu


stress (Sign_symptom) - Score: 0.64


In [53]:
specialist_mapping = {
    "chest": "Cardiologist",
    "shortness of breath": "Pulmonologist",
    "fever": "General Physician",
    "joint pain": "Orthopedic",
    "headache": "Neurologist",
    "stomach ache": "Gastroenterologist",
    "skin rash": "Dermatologist",
    "eye pain": "Ophthalmologist",
    "ear pain": "ENT Specialist",
}


In [21]:
def map_to_specialist(entities):
    specialist_map = {
        "Cardiologist": ["chest", "heart", "palpitation", "shortness of breath"],
        "Neurologist": ["headache", "seizure", "numbness", "dizziness"],
        "Gastroenterologist": ["stomach", "abdomen", "nausea", "diarrhea"],
        "Orthopedic": ["joint", "bone", "back", "fracture"],
        "Pulmonologist": ["cough", "lungs", "breathing", "asthma", "fever"],
        "Dermatologist": ["skin", "rash", "itching", "acne"],
        "Psychiatrist": ["depression", "anxiety", "sleep", "stress"],
    }

    # Prioritize based on entity_group and keywords
    for ent in entities:
        word = ent['word'].lower()
        label = ent['entity_group']

        if label in ["Sign_symptom", "Biological_structure"]:
            for specialist, keywords in specialist_map.items():
                for keyword in keywords:
                    if keyword in word:
                        return specialist

    return "General Physician"


In [22]:
map_to_specialist(entities)

'Pulmonologist'